In [0]:
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales 
import pandas as pd

#1. Leemos los datos de PROCEESED la tabla Delta 
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/processed/proyectocongestion_processed/datapreprocessed_congestion_tabladelta")
datos = df_delta.toPandas()

#2. Ajuste de los datos a Series Temporales
#2.1 Se establece la columna 'Time' como el índice del DataFrame "datos"
datos = datos.set_index('instant_date_t')

#2.3 Ordenamos el dataset de forma ascendente segun el datetime
datos.sort_index(inplace=True)

#2.4 Identificamos la periocidad de la serie temporal
df_time_diffs = datos.index.to_series().diff().dt.total_seconds()

#3. Eliminamos o Filtramos las filas donde la diferencia es distinta de cero (Con ello eliminamos las filas o registros de fechas duplicadas)
datos = datos[df_time_diffs != 0]

#4. # Reinterpolar el dataset con una periosidad en especifico
# 'S' o 'seg' : Segundo, 'T' o 'min': Minuto,'H' o 'Hr': Hora,'D': Día,'W': Semana,'M': Mes,'Q': Trimestre,'Y': Año
datos = datos.asfreq(freq='T', method='bfill')

datos.head()

# 5. Seleecion de variables en especifico luego de seleccion de caracteristicas (Correlacion, variables significaticas)
#datos = datos.loc[:, ['x', 'y']]

In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
datos.dtypes

In [0]:
datos['isload_t'].unique()

#### Guardar la tabla de caracteristicas en tabla delta

In [0]:
datos.head()

In [0]:
datos.dtypes

#### Pasos Adicionales: Nota (Estos procesos se deben hacer en un notebook por separado para no interunpir el flujo)

In [0]:
# Pasos Adicionales: Nota (Estos procesos se deben hacer en un notebook por separado para no interunpir el flujo)
# Ajustar la configuración para mostrar todas las columnas
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
# n.1 Listar todas las bases de datos
spark.sql("SHOW DATABASES").show(truncate=False)

In [0]:
# # n.2 Crear la base de datos si no existe en el almacenamiento de processed (en la ruta donde se almacenaran TablaDelta preprocesados)
spark.sql("CREATE DATABASE IF NOT EXISTS dbproyectocongestion_presentation LOCATION '/mnt/datalakemlopsd4m/presentation/proyectocongestion_presentation/'")

In [0]:
# 10. Convertir el DataFrame de Pandas a un DataFrame de Spark
spark_datos = spark.createDataFrame(datos)

# 11. Guardar los datos preprocesados en una tabla Delta en el Azure Storage 
# Asegurar que las columnas de fecha y hora mantengan sus tipos de datos
from pyspark.sql.functions import col
#spark_datos = spark_datos.withColumn("Event_Date", col("Event_Date").cast("timestamp"))
#spark_datos = spark_datos.withColumn("instant_date_t", col("instant_date_t").cast("timestamp"))

# Nombre de la tabla Delta a guardar
nombre_tabla_delta = "dbproyectocongestion_presentation.tablacaracteristicas_congestion_tabladelta"

# Verificar si ya existe la tabla Delta
if spark.catalog.tableExists(nombre_tabla_delta):
    # Eliminar la tabla Delta existente
    spark.sql("DROP TABLE IF EXISTS " + nombre_tabla_delta)

# Guardar los datos preprocesados en una tabla Delta
spark_datos.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(nombre_tabla_delta)